In [1]:
# 几个用来绘制灰度和 RGB 图像的函数
def plot_image(image):
    # "nearest" 分辨率不匹配时,只显示图像而不尝试在像素之间进行插值
    plt.imshow(image, cmap="gray", interpolation="nearest")
    plt.axis("off")

def plot_color_image(image):
    plt.imshow(image, interpolation="nearest")
    plt.axis("off")

In [2]:
import numpy as np
import pandas as pd
import tensorflow as tf
import tensorflow.keras as keras
import os
import time 
import matplotlib.pyplot as plt
import matplotlib as mpl
import seaborn as sns

from functools import partial

def initialization():
    keras.backend.clear_session()
    np.random.seed(42)
    tf.random.set_seed(42)

In [3]:
from matplotlib import font_manager
my_font = font_manager.FontProperties(fname='../Fonts/SourceHanSerifSC-Medium.otf', size=12)

# 分类和定位 Classification and Localization

`定位任务`可以表示为`分类任务`:预测物体周围的边界框,一种常用的方法是预测**物体中心的水平坐标和垂直坐标以及其高度和宽度**.

因此,可以添加具有四个单位的第二个`Dense层`,使用`MSE`损失对其进行训练.

1. 加载,拆分数据集

In [4]:
import tensorflow_datasets as tfds

dataset, info = tfds.load("tf_flowers", as_supervised=True, with_info=True)
class_names = info.features["label"].names
n_classes = info.features["label"].num_classes
dataset_size = info.splits["train"].num_examples

In [5]:
test_set_raw, valid_set_raw, train_set_raw = tfds.load(
    "tf_flowers",
    split=["train[:10%]", "train[10%:25%]", "train[25%:]"],
    as_supervised=True)

2. 图像预处理

In [6]:
# 中央裁切
def central_crop(image):
    shape = tf.shape(image)   # height, width
    min_dim = tf.reduce_min([shape[0], shape[1]])
    
    # 从图像左上到右下
    top_crop = (shape[0] - min_dim) // 4
    bottom_crop = shape[0] - top_crop
    
    left_crop = (shape[1] - min_dim) // 4
    right_crop = shape[1] - left_crop    
    
    return image[top_crop:bottom_crop, left_crop:right_crop]    

In [7]:
# 随机裁切
def random_crop(image):
    shape = tf.shape(image)
    min_dim = tf.reduce_min([shape[0], shape[1]]) * 90 // 100

    # 如果一个维度不应该被裁剪，则传递该维度的完整大小
    cropped_image = tf.image.random_crop(image, size=[min_dim, min_dim, 3])
    return cropped_image

In [8]:
def preprocess(image, label, randomize=False):
    if randomize:
        cropped_image = random_crop(image)
        # random_flip_left_right:随机水平翻转图像（从左到右）。
        cropped_image = tf.image.random_flip_left_right(cropped_image)
    else:
        cropped_image = central_crop(image)
    
    resized_image = tf.image.resize(cropped_image, [224, 224])
    final_image = keras.applications.xception.preprocess_input(resized_image)
    return final_image, label

In [9]:
batch_size = 32

train_set = train_set_raw.shuffle(1000).repeat()
train_set = train_set.map(partial(preprocess, randomize=True)).batch(batch_size).prefetch(1)

valid_set = valid_set_raw.map(preprocess).batch(batch_size).prefetch(1)

test_set = test_set_raw.map(preprocess).batch(batch_size).prefetch(1)

3. 加载预训练模型
    
    加载在 `ImageNet` 上预训练的` Xception` 模型

In [10]:
from tensorflow.keras.layers import Input, Dense, Conv2D, MaxPool2D, Softmax, BatchNormalization, ReLU, GlobalAvgPool2D, Add, Dropout
from tensorflow.keras.models import Model, Sequential

In [11]:
feature = keras.applications.Xception(include_top=False, weights="imagenet")

In [12]:
avg = GlobalAvgPool2D()(feature.output)
class_output = Dense(n_classes, activation="softmax")(avg)
loc_output = Dense(4)(avg) # 物体中心的水平坐标和垂直坐标以及其高度和宽度.
model = Model(inputs=feature.input, outputs=[class_output, loc_output])

默认情况下，`Keras` 将计算`loss`所有这些损失并简单地将它们相加以获得用于训练的最终损失。由于主输出比辅助输出（因为它只是用于正则化），所以我们想让主输出的损失更大,可以通过`loss_weights`设置.

In [13]:
optimizer = keras.optimizers.SGD(learning_rate=0.2, momentum=0.9, decay=0.01)

model.compile(loss=["sparse_categorical_crossentropy", "mse"],
              loss_weights=[0.8, 0.2],
              optimizer=optimizer,
              metrics=["accuracy"])

4. 随机生成边界框

    假设已经获得了`tf_flower`数据集中每个图像的边界框.接下来,需要创建一个数据集,其数据项是经过预处理的图像的批量处理以及其类标签和边界框.

    每个数据项形式如下:`(images, (class_lables,bounding_boxes))`
    
    对边界框应该进行归一化，以便水平和垂直坐标以及高度和宽度都在 0 到 1 的范围内。

    此外，通常要预测高度和宽度的平方根：对于,大边界框的 10 像素错误不会像对小边界框的 10 像素错误一样收到惩罚。

In [14]:
# 添加随机生成边界框
def add_random_bounding_boxes(images, lables):
    shape = tf.shape(images)   # [224, 224]
    fake_boxes = tf.random.uniform([shape[0], 4])   # [224, 4]
    return images, (lables, fake_boxes)

In [23]:
fake_train_set = train_set.take(5).repeat(2).map(add_random_bounding_boxes)

In [24]:
model.fit(fake_train_set, steps_per_epoch=5, epochs=2)

Epoch 1/2
5/5 [==============================] - 35s 6s/step - loss: 1.1612 - dense_loss: 1.3696 - dense_1_loss: 0.3274 - dense_accuracy: 0.4375 - dense_1_accuracy: 0.2000
Epoch 2/2
5/5 [==============================] - 29s 6s/step - loss: 0.5893 - dense_loss: 0.6893 - dense_1_loss: 0.1894 - dense_accuracy: 0.7437 - dense_1_accuracy: 0.2562


`MSE`通常作为成本函数训练效果很好，但它并不是评估模型预测边界框的好坏的一个很好的指标。最常见的度量指标是`交并比IoU`：预测边界框和目标边界框之间的重叠面积除以它们的联合面积。

在 `keras`中,由 `tf.keras.metrics.MeanIoU`类实现。

<img src="../images/other/14-53.png" width="400">
<img src="../images/other/14-54.png" width="200">

# 目标检测 Object Detection

对图像中的**多个对象进行分类和定位的任务**称为`目标检测`。
- 一种常见的方法是采用经过训练对单个对象进行分类和定位的 `CNN`，然后将其**在图像上滑动**.
    
    在此示例中，图像被切成 6 × 8 的网格，我们展示了一个 `CNN` 在所有 3 × 3 区域上滑动。当 `CNN` 查看图像的左上角时，它检测到最左边玫瑰的一部分，然后当它第一次向右移动一步时，它再次检测到同一朵玫瑰。在下一步，它开始检测最上面的玫瑰的一部分，然后一旦它向右移动了一步，它就会再次检测到它。然后，您将继续在整个图像中滑动`CNN`，查看所有 3 × 3 区域。此外，由于对象可以有不同的大小，您还可以在不同大小的区域之间滑动 CNN。
<img src="../images/other/14-55.png" width="300">



- 滑动操作,会在不同的位置多次检测到同一个对象。接下来需要进行一些**后期处理来消除所有不必要的边界框**。

    一种常见的方法称为`非极大抑制non-max suppression`。
    
    1. 首先，您需要在 CNN 中添加额外的 `置信度分数` 输出，以估计图像中存在花的概率。必须使用 `sigmoid` 激活函数，使用`二元交叉熵损失`来训练它。然后**删除所有 `置信度分数` 低于某个阈值的所有边界框**：这将删除所有实际上不包含花的边界框.
    2. **找到 `置信度分数` 最高的边界框，并删除所有其他与它重叠很多的边界框**（例如，`交并比IoU` ＞ 60%）。例如，上图中具有最大 `置信度分数` 分数的边界框是最顶部玫瑰上方的粗边界框 。同一朵玫瑰上的另一个边界框与最大边界框重叠很多，因此将其删除.
    3. 重复第二步,直到没有更多的边界框可以删除.

`目标检测`任务基本可以分成两类:

- Two-Stage:    (`Faster-RCNN`)
    1. 通过专门的模块去生成候选框(RPN),寻找前景以及调整边界框(`基于anchors`)
    2. 基于之前生成的候选框进一步分类和调整候选框(`基于proposals`)
    3. 优点:检测更准确
    
    
- One-Stage:    (`SSD`,`YOLO`)
    1. `基于 anchors`直接进行分类以及调整边界框
    2. 优点:检测速度快

这种简单的目标检测方法效果很好，但它需要多次运行CNN，因此速度很慢。幸运的是，有一种更快的方法可以在图像上滑动 CNN：`完全卷积网络FCN`.

## 全卷积网络 Fully Convolutional Networks

> create:Apotosome 

> update:Apotosome 10/26/22